<a href="https://colab.research.google.com/github/fabiormazza/IA025_2022S1/blob/main/projeto_final/fabio_mazza/Hipotese_Ws_WinnerImproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.1 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import tqdm

from collections import OrderedDict


In [3]:
# Check which GPU we are using
!nvidia-smi

Tue Jun 21 23:35:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [5]:
from typing import List


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        
        self.max_seq_length = max_seq_length
        cls = 101
        sep = 100
        tokens_all = torch.tensor([])
        inputs = torch.tensor([])
        targets = torch.tensor([])
 
        for text in tqdm.notebook.tqdm(texts):
            tokens = tokenize(text, tokenizer)
            tokens = [cls] + tokens + [sep]
            tokens = torch.FloatTensor(tokens)
            tokens_all = torch.cat((tokens_all, tokens))
            #print(tokens_all)
        
        pad_size = max_seq_length - tokens_all.size(dim=0) % max_seq_length
        tokens_all = torch.cat((tokens_all, torch.zeros(pad_size)), dim=0) #pad_token_id = 0 (review if otherwise) / tokens_uns is the tokens tensor before reshaping
        inputs = tokens_all.reshape((-1, max_seq_length))

        t_tokens = torch.roll(tokens_all, -1, 0)
        t_tokens[-1] = 0
        targets = t_tokens.reshape((-1, max_seq_length))
        
        self.inputs = torch.LongTensor(inputs.numpy())
        self.targets = torch.LongTensor(targets.numpy())

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [6]:
## Teste com frases longas

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Os primeiros socorros foram feitos no local e a vítima foi levada \
ao Hospital de Base de Brasília com ferimentos leves, consciente e com estado de saúde estável.', 
'Um veículo capotou na Via Estrutural na tarde desta sexta-feira (3), por volta das 15h. O Corpo\
 de Bombeiros Militar do Distrito Federal (CBMDF) foi chamado ao local para realizar os primeiros atendimentos à vítima.']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)

first_batch_input, first_batch_target = next(iter(dummy_loader))

print(first_batch_input)
print(first_batch_target)

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([[  101,   533,  1867, 19279,   444,   506,  5159,   202,  1238],
        [  122,   123,  8329,   262, 11067,   320,  6297,   125,  8831],
        [  125,  6191,   170, 16811, 14698,   117, 19054,   122,   170],
        [ 1177,   125,  3231, 12055,   119,   100,   101,  1263,  7594],
        [  853,  3356,   203,   229,  7811,  9009,   381,  1441,   229],
        [ 1373,  1014,  7250,   118, 14258,   113,   511,   114,   117]])
tensor([[  533,  1867, 19279,   444,   506,  5159,   202,  1238,   122],
        [  123,  8329,   262, 11067,   320,  6297,   125,  8831,   125],
        [ 6191,   170, 16811, 14698,   117, 19054,   122,   170,  1177],
        [  125,  3231, 12055,   119,   100,   101,  1263,  7594,   853],
        [ 3356,   203,   229,  7811,  9009,   381,  1441,   229,  1373],
        [ 1014,  7250,   118, 14258,   113,   511,   114,   117,   240]])


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [7]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-21 23:35:46--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 74.125.203.128, 2404:6800:4008:c13::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   128MB/s    in 9.9s    

2022-06-21 23:35:57 (118 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [8]:
# Load datasets
max_seq_length = 9

train_examples = 10000
valid_examples = 2000
test_examples = 2000

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 14000 lines.


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [9]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 1318908
valid examples: 245865
test examples: 231629


In [17]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super(LanguageModel, self).__init__()

        self.V = vocab_size
        self.L = max_seq_length
        self.D = dim
        self.n_layers = n_layers
        self.pad_token_id = pad_token_id

        n_linear = 512

        self.embedding = torch.nn.Embedding(vocab_size, dim, device=device)
        self.positional = torch.nn.Parameter(torch.randn(max_seq_length, dim, device=device)/1000000000)
        
        self.wA = nn.Linear(dim, dim, bias = False, device=device)
        self.wB = nn.Linear(dim, dim, bias = False, device=device)
        self.wC = nn.Linear(dim, dim, bias = False, device=device)

        self.wD = nn.Linear(dim, dim, bias = False, device=device)
        self.wE = nn.Linear(dim, dim, bias = False, device=device)
        self.wF = nn.Linear(dim, dim, bias = False, device=device)

        self.wG = nn.Linear(dim, dim, bias = False, device=device)
        self.wH = nn.Linear(dim, dim, bias = False, device=device)
        self.wI = nn.Linear(dim, dim, bias = False, device=device)

        self.wJ = nn.Linear(dim, dim, bias = False, device=device)
        self.wK = nn.Linear(dim, dim, bias = False, device=device)
        self.wL = nn.Linear(dim, dim, bias = False, device=device)



        #self.wq = nn.Linear(dim, dim, bias = False, device=device)
        #self.wk = nn.Linear(dim, dim, bias = False, device=device)
        #self.wv = nn.Linear(dim, dim, bias = False, device=device)
        #self.wo = nn.Linear(dim, dim, bias = False, device=device)

        self.linear1 = nn.Sequential(OrderedDict([
                              ('l11', torch.nn.Linear(4 * dim, n_linear, device=device)),
                              ('relu1', torch.nn.ReLU()),
                              ('dropout1', torch.nn.Dropout(p=0.20)),
                              ('l12', torch.nn.Linear(n_linear, 4 * dim, device=device, bias = False))
        ]))

        self.linear2 = nn.Sequential(OrderedDict([
                              ('l21', torch.nn.Linear(4 * dim, n_linear, device=device)),
                              ('relu2', torch.nn.ReLU()),
                              ('dropout2', torch.nn.Dropout(p=0.20)),
                              ('l22', torch.nn.Linear(n_linear, 4 * dim, device=device, bias = False))
        ]))

        self.linear3 = nn.Sequential(OrderedDict([
                              ('l31', torch.nn.Linear(4 * dim, n_linear, device=device)),
                              ('relu3', torch.nn.ReLU()),
                              ('dropout3', torch.nn.Dropout(p=0.20)),
                              ('l32', torch.nn.Linear(n_linear, vocab_size, device=device, bias = False))
        ]))

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        X = self.embedding(inputs)
        X = X + self.positional
        #Q = self.wq(X)
        #K = self.wk(X)
        #V = self.wv(X)

        A = self.wA(X)
        B = self.wB(X)
        C = self.wC(X)

        D = self.wD(X)
        E = self.wE(X)
        F = self.wF(X)

        G = self.wG(X)
        H = self.wH(X)
        I = self.wI(X)

        J = self.wJ(X)
        K = self.wK(X)
        L = self.wL(X)

        AB = torch.matmul(A, B.permute(0, 2, 1))/math.sqrt(self.D)
        DE = torch.matmul(D, E.permute(0, 2, 1))/math.sqrt(self.D)
        GH = torch.matmul(G, H.permute(0, 2, 1))/math.sqrt(self.D)
        JK = torch.matmul(J, K.permute(0, 2, 1))/math.sqrt(self.D)

        #causal mask
        mask = torch.ones((self.L, self.L), dtype=torch.bool)
        mask = torch.triu(mask, diagonal=1)
        AB[:, mask] = -1000000000
        DE[:, mask] = -1000000000
        GH[:, mask] = -1000000000
        JK[:, mask] = -1000000000

        AB_probs = torch.nn.functional.softmax(AB, dim=-1)
        DE_probs = torch.nn.functional.softmax(DE, dim=-1)
        GH_probs = torch.nn.functional.softmax(GH, dim=-1)
        JK_probs = torch.nn.functional.softmax(JK, dim=-1)

        M = self.wC(torch.matmul(AB_probs, C))
        N = self.wF(torch.matmul(DE_probs, F))
        O = self.wI(torch.matmul(GH_probs, I))
        P = self.wL(torch.matmul(JK_probs, L))

        MNOP = torch.cat((M, N, O, P), dim=2)
 

        out = self.linear1(MNOP)
        out = out + MNOP
        residual = out
        out = self.linear2(out)
        out = out + residual
        out = self.linear3(out)

        return out

## Teste o modelo com um exemplo

In [18]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [19]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 17867968


## Assert da Perplexidade


In [20]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              29925
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 2_000_000 #150_000_000
eval_every_steps = 1000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=128,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=256, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=256)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29884.80, valid ppl: 29511.14
1000 steps; 256000 examples so far; train ppl: 1145.69, valid ppl: 707.00
2000 steps; 512000 examples so far; train ppl: 593.51, valid ppl: 482.77
3000 steps; 768000 examples so far; train ppl: 444.06, valid ppl: 390.83
4000 steps; 1024000 examples so far; train ppl: 379.18, valid ppl: 347.50
5000 steps; 1280000 examples so far; train ppl: 342.62, valid ppl: 320.45
6000 steps; 1536000 examples so far; train ppl: 303.29, valid ppl: 304.18


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=256)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz muito'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

In [ ]:
prompt = 'A elevação nos valores dos combustíveis é vista como '
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

In [ ]:
prompt = 'A viagem partirá de Campinas por meio ferroviário e'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.